In [ ]:
import duckdb
import geopandas as gpd
import pandas as pd
from h3pandas.util.shapely import polyfill
import pyarrow.dataset as ds
from lonboard import viz, Map, SolidPolygonLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.diverging import BrBG_10
from palettable.cartocolors.diverging import Geyser_3, TealRose_7
import numpy as np

In [ ]:
con = duckdb.connect()

dataset = ds.dataset("../h3_4_2022_deep/", format="parquet")
con.register("dataset", dataset)

df = con.execute("""
	select cell as h3, AphiaID
    from dataset
""").fetchdf()
df

In [ ]:
con = duckdb.connect()
con.register("df", df)

result = con.execute("""
	select
        h3, count(distinct(AphiaID)) as species
    from df
    group by h3
""").fetchdf()
result

In [ ]:
result_poly = result.set_index("h3").h3.h3_to_geo_boundary()
offending = list(result_poly.cx[179:180, -90:90].index) + list(result_poly.cx[-180:-179, -90:90].index)
result_poly = result_poly.loc[result_poly.index.difference(offending), :]

In [ ]:
normalized_species = np.log(result_poly["species"]) / np.log(result_poly["species"]).max()
normalized_species
color = apply_continuous_cmap(normalized_species, TealRose_7, alpha=0.7)
color

In [ ]:
polygon_layer = SolidPolygonLayer.from_geopandas(
    result_poly,
    get_fill_color=color,
    opacity=0.9
)
map = Map([polygon_layer])
map

In [ ]:
with open("/Users/pieter/Desktop/indicator.html", "w") as f:
    f.write(map.as_html().data)